In [23]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [33]:
%cd "/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier"

/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier


In [1]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

# Loading Raw Data

In [2]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:, 0:27, 0:27]
x_test = x_test[:, 0:27, 0:27]

In [3]:
x_train_flatten = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/255.0
x_test_flatten = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255.0

In [4]:
print(x_train_flatten.shape, y_train.shape)
print(x_test_flatten.shape, y_test.shape)

(60000, 729) (60000,)
(10000, 729) (10000,)


In [5]:
x_train_0 = x_train_flatten[y_train == 0]
x_train_1 = x_train_flatten[y_train == 1]
x_train_2 = x_train_flatten[y_train == 2]
x_train_3 = x_train_flatten[y_train == 3]
x_train_4 = x_train_flatten[y_train == 4]
x_train_5 = x_train_flatten[y_train == 5]
x_train_6 = x_train_flatten[y_train == 6]
x_train_7 = x_train_flatten[y_train == 7]
x_train_8 = x_train_flatten[y_train == 8]
x_train_9 = x_train_flatten[y_train == 9]

x_train_list = [x_train_0, x_train_1, x_train_2, x_train_3, x_train_4, x_train_5, x_train_6, x_train_7, x_train_8, x_train_9]

print(x_train_0.shape)
print(x_train_1.shape)
print(x_train_2.shape)
print(x_train_3.shape)
print(x_train_4.shape)
print(x_train_5.shape)
print(x_train_6.shape)
print(x_train_7.shape)
print(x_train_8.shape)
print(x_train_9.shape)

(5923, 729)
(6742, 729)
(5958, 729)
(6131, 729)
(5842, 729)
(5421, 729)
(5918, 729)
(6265, 729)
(5851, 729)
(5949, 729)


In [6]:
x_test_0 = x_test_flatten[y_test == 0]
x_test_1 = x_test_flatten[y_test == 1]
x_test_2 = x_test_flatten[y_test == 2]
x_test_3 = x_test_flatten[y_test == 3]
x_test_4 = x_test_flatten[y_test == 4]
x_test_5 = x_test_flatten[y_test == 5]
x_test_6 = x_test_flatten[y_test == 6]
x_test_7 = x_test_flatten[y_test == 7]
x_test_8 = x_test_flatten[y_test == 8]
x_test_9 = x_test_flatten[y_test == 9]

x_test_list = [x_test_0, x_test_1, x_test_2, x_test_3, x_test_4, x_test_5, x_test_6, x_test_7, x_test_8, x_test_9]

print(x_test_0.shape)
print(x_test_1.shape)
print(x_test_2.shape)
print(x_test_3.shape)
print(x_test_4.shape)
print(x_test_5.shape)
print(x_test_6.shape)
print(x_test_7.shape)
print(x_test_8.shape)
print(x_test_9.shape)

(980, 729)
(1135, 729)
(1032, 729)
(1010, 729)
(982, 729)
(892, 729)
(958, 729)
(1028, 729)
(974, 729)
(1009, 729)


# Selecting the dataset

Output: X_train, Y_train, X_test, Y_test

In [7]:
num_sample = 200
n_class = 4
mult_test = 0.25

X_train = x_train_list[0][:num_sample, :]
X_test = x_test_list[0][:int(mult_test*num_sample), :]

Y_train = np.zeros((n_class*X_train.shape[0],), dtype=int)
Y_test = np.zeros((n_class*X_test.shape[0],), dtype=int)

for i in range(n_class-1):
  X_train = np.concatenate((X_train, x_train_list[i+1][:num_sample, :]), axis=0)
  Y_train[num_sample*(i+1):num_sample*(i+2)] = int(i+1)

  X_test = np.concatenate((X_test, x_test_list[i+1][:int(mult_test*num_sample), :]), axis=0)
  Y_test[int(mult_test*num_sample*(i+1)):int(mult_test*num_sample*(i+2))] = int(i+1)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(800, 729) (800,)
(200, 729) (200,)


# Dataset Preprocessing

In [8]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((800, 27, 27), (200, 27, 27))

# Quantum

In [9]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

qml.enable_tape()

from tensorflow.keras.utils import to_categorical

# Set a random seed
np.random.seed(2020)

In [10]:
# Define output labels as quantum state vectors

# def density_matrix(state):
#     """Calculates the density matrix representation of a state.

#     Args:
#         state (array[complex]): array representing a quantum state vector

#     Returns:
#         dm: (array[complex]): array representing the density matrix
#     """
#     return state * np.conj(state).T


label_0 = [[1], [0]]
label_1 = [[0], [1]]


def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return np.outer(state,  np.conj(state))

#state_labels = [label_0, label_1]
state_labels = np.loadtxt('./tetra_states.txt', dtype=np.complex_)

In [11]:
dm_labels = [density_matrix(state_labels[i]) for i in range(4)]
len(dm_labels)

4

In [12]:
dm_labels

[tensor([[1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]], requires_grad=True),
 tensor([[0.33333333+0.j, 0.47140452+0.j],
         [0.47140452+0.j, 0.66666667+0.j]], requires_grad=True),
 tensor([[ 0.33333333+0.j        , -0.23570226-0.40824829j],
         [-0.23570226+0.40824829j,  0.66666667+0.j        ]], requires_grad=True),
 tensor([[ 0.33333333+0.j        , -0.23570226+0.40824829j],
         [-0.23570226-0.40824829j,  0.66666667+0.j        ]], requires_grad=True)]

In [13]:
n_qubits = 4  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs = [x, y]
        x (array[float]): 1-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]


In [14]:
dev_conv = qml.device("default.qubit", wires=9)


@qml.qnode(dev_conv)
def q_conv(conv_params, inputs):
    """A variational quantum circuit representing the Universal classifier + Conv.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    # layer iteration
    for l in range(len(conv_params[0])):
        # RY layer
        # height iteration
        for i in range(3):
            # width iteration
            for j in range(3):
                qml.RY((conv_params[0][l][3*i+j] * inputs[i, j] + conv_params[1][l][3*i+j]), wires=(3*i+j))

        # entangling layer
        for i in range(9):
            if i != (9-1):
                qml.CNOT(wires=[i, i+1])
                
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3) @ qml.PauliZ(4) @ qml.PauliZ(5) @ qml.PauliZ(6) @ qml.PauliZ(7) @ qml.PauliZ(8))

In [15]:
a = np.zeros((2, 1, 9))
q_conv(a, X_train[0, 0:3, 0:3])

tensor(1., requires_grad=True)

In [16]:
a = np.zeros((2, 1, 9))
q_fc(a, X_train[0, 0, 0:9])

tensor([1.        , 0.33333333, 0.33333333, 0.33333333], requires_grad=True)

In [17]:
tetra_class = np.loadtxt('./tetra_class_label.txt')
binary_class = np.array([[1, 0], [0, 1]])

square_class = np.array(np.loadtxt('./square_class_label.txt', dtype=np.complex_), dtype=float)

/home/ta-edo/.local/lib/python3.8/site-packages/autograd/numpy/numpy_wrapper.py:73: ComplexWarning: Casting complex values to real discards the imaginary part
  return _np.array(scalar, *array_args, **array_kwargs)


In [18]:
class_labels = tetra_class

In [19]:
class_labels

tensor([[1.        , 0.33333333, 0.33333333, 0.33333333],
        [0.33333333, 1.        , 0.33333333, 0.33333333],
        [0.33333333, 0.33333333, 1.        , 0.33333333],
        [0.33333333, 0.33333333, 0.33333333, 1.        ]], requires_grad=True)

In [20]:
n_class = 4

temp = np.zeros((len(Y_train), n_class))
for i in range(len(Y_train)):
    temp[i, :] = class_labels[Y_train[i]]
Y_train = temp


temp = np.zeros((len(Y_test), n_class))
for i in range(len(Y_test)):
    temp[i, :] = class_labels[Y_test[i]]
Y_test = temp

In [21]:
Y_test

tensor([[1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.        , 0.33333333, 0.33333333, 0.33333333],
        [1.   

In [22]:
from keras import backend as K


# Alpha Custom Layer
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, n_class), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [23]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 2
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 2
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18*L*n_class + 2, output size = 2
num_fc_layer = 2
q_fc_layer_0 = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=n_class)(reshape_layer_3)

# Alpha Layer
alpha_layer_0 = class_weights()(q_fc_layer_0)

model = tf.keras.Model(inputs=X, outputs=alpha_layer_0)

In [24]:
model(X_train[0:32, :, :])

<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[ 0.0247839 ,  0.01879054, -0.00123031, -0.01002374],
       [ 0.02476473,  0.01873888, -0.00123847, -0.00994578],
       [ 0.02472481,  0.018805  , -0.00124422, -0.00978129],
       [ 0.02470079,  0.01880255, -0.0012503 , -0.00968586],
       [ 0.02473471,  0.01882366, -0.00124057, -0.00982054],
       [ 0.02475395,  0.01872398, -0.00124211, -0.00990245],
       [ 0.02481924,  0.0186645 , -0.0012294 , -0.01017524],
       [ 0.02474313,  0.01886819, -0.0012356 , -0.00985379],
       [ 0.02477125,  0.01877953, -0.00123422, -0.00997135],
       [ 0.02478937,  0.01873251, -0.00123265, -0.01004783],
       [ 0.02476793,  0.01875675, -0.00123652, -0.0099583 ],
       [ 0.02473665,  0.01890324, -0.00123496, -0.00982683],
       [ 0.02476653,  0.01873967, -0.00123796, -0.00995314],
       [ 0.02470708,  0.01884012, -0.00124637, -0.00970903],
       [ 0.02477513,  0.01876597, -0.00123411, -0.00998775],
       [ 0.02475806,  0.0189284 , -0

In [25]:
import keras.backend as K

# def custom_loss(y_true, y_pred):
#     return K.sum(((y_true.shape[1]-2)*y_true+1)*K.square(y_true-y_pred))/len(y_true)

def custom_loss(y_true, y_pred):
    loss = K.square(y_true-y_pred)
    #class_weights = y_true*(weight_for_1-weight_for_0) + weight_for_0
    #loss = loss * class_weights
    
    return K.sum(loss)/len(y_true)

In [26]:
for i in range(10):
    print(0.1* ((0.95)**i))

0.1
0.095
0.09025
0.0857375
0.081450625
0.07737809374999999
0.07350918906249998
0.06983372960937498
0.06634204312890622
0.0630249409724609


In [27]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=int(len(X_train)/32),
    decay_rate=0.95,
    staircase=True)

In [28]:
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [29]:
cp_val_acc = tf.keras.callbacks.ModelCheckpoint(filepath="./Model/4_QConv2ent_QFC_LRDecay_valacc.hdf5",
                monitor='val_accuracy', verbose=1, save_weights_only=True, save_best_only=True, mode='max')

cp_val_loss = tf.keras.callbacks.ModelCheckpoint(filepath="./Model/4_QConv2ent_QFC_LRDecay_valloss.hdf5",
                monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

In [ ]:
H = model.fit(X_train, Y_train, epochs=15, batch_size=32, initial_epoch=13,
              validation_data=(X_test, Y_test), verbose=1,
              callbacks=[cp_val_acc, cp_val_loss])

Epoch 14/15
25/25 [==============================] - 26759s 1071s/step - loss: 0.0365 - accuracy: 0.7900 - val_loss: 0.0419 - val_accuracy: 0.7750

Epoch 00014: val_accuracy improved from 0.77000 to 0.77500, saving model to ./Model/4_QConv2ent_QFC_LRDecay_valacc.hdf5

Epoch 00014: val_loss improved from 0.04191 to 0.04188, saving model to ./Model/4_QConv2ent_QFC_LRDecay_valloss.hdf5
Epoch 15/15
 4/25 [===>..........................] - ETA: 6:04:18 - loss: 0.0328 - accuracy: 0.8750

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 27, 27)]     0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 3, 3)         0           Input_Layer[0][0]                
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 3, 3)         0           Input_Layer[0][0]                
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 3, 3)         0           Input_Layer[0][0]                
______________________________________________________________________________________________

In [32]:
# best first 10 epochs
H.history

{'loss': [0.15218332409858704,
  0.0771825984120369,
  0.06443765759468079,
  0.055341072380542755,
  0.04958663508296013,
  0.049124058336019516,
  0.043934889137744904,
  0.046720340847969055,
  0.04242421314120293,
  0.03936174511909485],
 'accuracy': [0.28999999165534973,
  0.46875,
  0.5950000286102295,
  0.6462500095367432,
  0.7087500095367432,
  0.7237499952316284,
  0.7637500166893005,
  0.7325000166893005,
  0.7737500071525574,
  0.7950000166893005],
 'val_loss': [0.08642026036977768,
  0.07997884601354599,
  0.056794241070747375,
  0.05375337228178978,
  0.0515001118183136,
  0.05256782844662666,
  0.06304561346769333,
  0.05081057548522949,
  0.048174671828746796,
  0.041906096041202545],
 'val_accuracy': [0.49000000953674316,
  0.4950000047683716,
  0.6050000190734863,
  0.6700000166893005,
  0.6850000023841858,
  0.7149999737739563,
  0.6549999713897705,
  0.7049999833106995,
  0.675000011920929,
  0.7699999809265137]}

In [36]:
# epoch 11-13
H.history

{'loss': [0.03849232941865921, 0.03880036994814873, 0.03897182643413544],
 'accuracy': [0.8125, 0.7975000143051147, 0.8125],
 'val_loss': [0.0439734011888504, 0.045246705412864685, 0.042141810059547424],
 'val_accuracy': [0.7149999737739563, 0.7049999833106995, 0.7450000047683716]}

In [33]:
# best first 10 epochs weights
model.weights

[<tf.Variable 'model/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 2, 9) dtype=float32, numpy=
 array([[[ 2.4900079 ,  1.4090514 , -1.5159504 , -0.4848129 ,
          -0.07513718,  0.41361067,  0.32082155, -0.02647311,
           1.4427327 ],
         [ 1.4590902 , -0.20396537, -1.3288578 ,  0.51830167,
          -0.36024192,  0.43985152, -0.46546036, -0.09139597,
          -1.601005  ]],
 
        [[ 0.07628264,  0.8245568 , -0.7381987 ,  0.6701282 ,
           0.02025853, -0.16835278, -0.4134181 , -0.14421725,
           0.49609786],
         [-0.07567957,  0.0277558 , -0.3067742 , -0.46371996,
          -0.05226158, -0.3737983 ,  0.4056428 , -0.26080084,
          -0.5278124 ]]], dtype=float32)>,
 <tf.Variable 'model/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 2, 9) dtype=float32, numpy=
 array([[[-0.05883733,  0.34953204,  1.0755701 ,  1.6524311 ,
           0.54020005, -0.16705374, -2.5600445 ,  1.1857815 ,
           0.44120708],
         [ 0.05490269,  0.35323524,  1.09177   ,

# Exploring the results

In [26]:
X_train = np.concatenate((x_train_list[0][:20, :], x_train_list[1][:20, :]), axis=0)
Y_train = np.zeros((X_train.shape[0],), dtype=int)
Y_train[20:] += 1

X_train.shape, Y_train.shape

((40, 729), (40,))

In [27]:
X_test = np.concatenate((x_test_list[0][:20, :], x_test_list[1][:20, :]), axis=0)
Y_test = np.zeros((X_test.shape[0],), dtype=int)
Y_test[20:] += 1

X_test.shape, Y_test.shape

((40, 729), (40,))

In [28]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((40, 27, 27), (40, 27, 27))

## First Layer

In [29]:
qconv_1_weights = np.array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]])

qconv_1_weights.shape

(2, 1, 9)

In [30]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)

qconv1_model = tf.keras.Model(inputs=X, outputs=reshape_layer_1)


In [31]:
qconv1_model(X_train[0:1])
qconv1_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])

In [32]:
qconv1_model.weights

[<tf.Variable 'model_1/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]], dtype=float32)>]

In [33]:
preprocessed_img_train = qconv1_model(X_train)

preprocessed_img_test = qconv1_model(X_test)

data_train = preprocessed_img_train.numpy().reshape(-1, 13*13)
np.savetxt('./2_QConv2ent_QFC-Filter1_Image_Train.txt', data_train)

data_test = preprocessed_img_test.numpy().reshape(-1, 13*13)
np.savetxt('./2_QConv2ent_QFC-Filter1_Image_Test.txt', data_test)

print(data_train.shape, data_test.shape)

(40, 169) (40, 169)


## Second Layer

In [34]:
qconv_2_weights = np.array([[[ 2.531285  , -0.53907233,  1.2279214 , -0.04873934,
          -1.9625113 , -0.31179625,  1.4537607 , -0.3272767 ,
           1.5368826 ]],
 
        [[ 0.8165296 ,  0.3640772 ,  1.7864715 ,  0.5245018 ,
          -1.6975542 , -0.21682522,  2.097541  ,  0.49968052,
           2.2059586 ]]])

qconv_2_weights.shape

(2, 1, 9)

In [35]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)

qconv2_model = tf.keras.Model(inputs=X, outputs=reshape_layer_2)


In [36]:
qconv2_model(X_train[0:1])
qconv2_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])
qconv2_model.get_layer('Quantum_Conv_Layer_2').set_weights([qconv_2_weights])

In [37]:
qconv2_model.weights

[<tf.Variable 'model_2/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 1.6704171 , -0.37210035, -1.4763509 , -0.09573141,
          -1.4124248 , -0.5384489 , -1.5873678 , -0.4178921 ,
           0.86764395]],
 
        [[ 0.07841697,  0.06181896, -0.11658821,  0.08088297,
          -0.22606611, -0.47507533,  0.20372277,  0.43287253,
          -0.4662246 ]]], dtype=float32)>,
 <tf.Variable 'model_2/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 2.531285  , -0.53907233,  1.2279214 , -0.04873934,
          -1.9625113 , -0.31179625,  1.4537607 , -0.3272767 ,
           1.5368826 ]],
 
        [[ 0.8165296 ,  0.3640772 ,  1.7864715 ,  0.5245018 ,
          -1.6975542 , -0.21682522,  2.097541  ,  0.49968052,
           2.2059586 ]]], dtype=float32)>]

In [38]:
preprocessed_img_train = qconv2_model(X_train)

preprocessed_img_test = qconv2_model(X_test)

data_train = preprocessed_img_train.numpy().reshape(-1, 6*6)
np.savetxt('./2_QConv2ent_QFC-Filter2_Image_Train.txt', data_train)

data_test = preprocessed_img_test.numpy().reshape(-1, 6*6)
np.savetxt('./2_QConv2ent_QFC-Filter2_Image_Test.txt', data_test)

print(data_train.shape, data_test.shape)

(40, 36) (40, 36)


## Quantum States

In [39]:
q_fc_weights = np.array([[[ 0.3726357 , -1.0686979 ,  0.24734715,  2.5706375 ,
           3.4816115 , -0.3167716 ,  1.2021828 ,  0.19082105,
           0.0209959 ]],
 
        [[-0.213224  ,  1.235254  ,  2.298314  ,  2.2815244 ,
          -1.1340573 ,  0.8286312 ,  0.6926544 , -1.0297745 ,
           0.14662777]]])

q_fc_weights.shape

(2, 1, 9)

In [40]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [41]:
np.argmax(pred_train, axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], requires_grad=True)

In [42]:
np.argmax(pred_test, axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], requires_grad=True)

In [43]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)

maxpool_model = tf.keras.Model(inputs=X, outputs=reshape_layer_3)


In [44]:
maxpool_model(X_train[0:1])
maxpool_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])
maxpool_model.get_layer('Quantum_Conv_Layer_2').set_weights([qconv_2_weights])

In [45]:
maxpool_train = maxpool_model(X_train)
maxpool_test = maxpool_model(X_test)

maxpool_train.shape, maxpool_test.shape

(TensorShape([40, 9]), TensorShape([40, 9]))

In [46]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [47]:
q_fc_state(np.zeros((2,1,9)), maxpool_train[0])

tensor(1., requires_grad=True)

In [48]:
q_fc_state(q_fc_weights, maxpool_train[0])

tensor(0.95763762, requires_grad=True)

In [49]:
train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

for i in range(len(train_state)):
    
    q_fc_state(q_fc_weights, maxpool_train[i])
    temp = np.flip(dev_state._state)
    train_state[i, :] = temp
    
    q_fc_state(q_fc_weights, maxpool_test[i])
    temp = np.flip(dev_state._state)
    test_state[i, :] = temp

In [50]:
# sanity check

print(((np.conj(train_state) @ density_matrix(state_labels[0])) * train_state)[:, 0] > 0.5)

print(((np.conj(test_state) @ density_matrix(state_labels[0])) * test_state)[:, 0] > 0.5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False False False
 False  True False False False False False False False False False False
 False False False False]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False False False]


In [51]:
np.savetxt('./2_QConv2ent_QFC-State_Train.txt', train_state)
np.savetxt('./2_QConv2ent_QFC-State_Test.txt', test_state)

## Random Starting State

In [84]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18*L*n_class + 2, output size = 2
num_fc_layer = 1
q_fc_layer_0 = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=2)(reshape_layer_3)

# Alpha Layer
alpha_layer_0 = class_weights()(q_fc_layer_0)

model_random = tf.keras.Model(inputs=X, outputs=alpha_layer_0)
model_maxpool_random = tf.keras.Model(inputs=X, outputs=reshape_layer_3)

In [85]:
model_random(X_train[0:1])
model_random.weights

[<tf.Variable 'model_8/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.4889698 ,  0.2380408 ,  0.09439629, -0.26716974,
           0.02344316, -0.06644684, -0.3357206 ,  0.3326074 ,
          -0.06934038]],
 
        [[ 0.3830526 ,  0.23494643,  0.41627997,  0.35118043,
          -0.42000604,  0.25128657,  0.5226568 , -0.27474615,
          -0.485173  ]]], dtype=float32)>,
 <tf.Variable 'model_8/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.12407106,  0.4605006 , -0.05004978, -0.00949878,
          -0.4083493 ,  0.18536532, -0.2448183 , -0.32981884,
          -0.38940096]],
 
        [[-0.45641547, -0.41075683,  0.4251256 , -0.17653632,
           0.01394123,  0.12382543, -0.23449615,  0.47272468,
           0.31813556]]], dtype=float32)>,
 <tf.Variable 'model_8/keras_layer_4/params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.4163184 ,  0.29198825,  0.49920654,  0.33594978,
          -0.49212807, 

In [86]:
random_weights = np.array([[[-0.4163184 ,  0.29198825,  0.49920654,  0.33594978,
          -0.49212807,  0.00343066,  0.30105686, -0.15320912,
          -0.5011647 ]],
 
        [[ 0.41882396,  0.17975801,  0.3508029 ,  0.37545007,
          -0.37378743,  0.39107925, -0.3128681 , -0.22416279,
          -0.00185567]]])

In [87]:
maxpool_train = model_maxpool_random(X_train)
maxpool_test = model_maxpool_random(X_test)

maxpool_train.shape, maxpool_test.shape

(TensorShape([40, 9]), TensorShape([40, 9]))

In [88]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [89]:
q_fc_state(np.zeros((2,1,9)), maxpool_train[0])

tensor(1., requires_grad=True)

In [92]:
q_fc_state(random_weights, maxpool_train[21])

tensor(0.75178797, requires_grad=True)

In [93]:
train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

for i in range(len(train_state)):
    
    q_fc_state(random_weights, maxpool_train[i])
    temp = np.flip(dev_state._state)
    train_state[i, :] = temp
    
    q_fc_state(random_weights, maxpool_test[i])
    temp = np.flip(dev_state._state)
    test_state[i, :] = temp

In [94]:
# sanity check

print(((np.conj(train_state) @ density_matrix(state_labels[0])) * train_state)[:, 0] > 0.5)

print(((np.conj(test_state) @ density_matrix(state_labels[0])) * test_state)[:, 0] > 0.5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [95]:
np.savetxt('./2_QConv2ent_QFC-RandomState_Train.txt', train_state)
np.savetxt('./2_QConv2ent_QFC-RandomState_Test.txt', test_state)

# Finish

In [76]:
first_10_epoch = H.history
first_10_epoch

{'loss': [0.7249730825424194,
  0.485493928194046,
  0.2910504639148712,
  0.20750370621681213,
  0.1200886219739914,
  0.09333930909633636,
  0.09352166950702667,
  0.09991505742073059,
  0.0711403489112854,
  0.0675935298204422],
 'accuracy': [0.5249999761581421,
  0.6225000023841858,
  0.7825000286102295,
  0.8650000095367432,
  0.9449999928474426,
  0.9599999785423279,
  0.9624999761581421,
  0.9524999856948853,
  0.9750000238418579,
  0.9775000214576721],
 'val_loss': [0.6175757646560669,
  0.40504130721092224,
  0.3411727547645569,
  0.19497469067573547,
  0.11362368613481522,
  0.07933302223682404,
  0.1450268030166626,
  0.0960061326622963,
  0.09231595695018768,
  0.053687386214733124],
 'val_accuracy': [0.5,
  0.7300000190734863,
  0.7400000095367432,
  0.8700000047683716,
  0.9449999928474426,
  0.9800000190734863,
  0.925000011920929,
  0.9649999737739563,
  0.9549999833106995,
  0.9750000238418579]}

In [77]:
# initial 10 epoch
model.get_weights()

[array([[[ 0.13920915, -0.7933977 ,  0.4931736 , -0.34984022,
           0.41861725, -0.1258272 , -0.3323837 ,  1.4861933 ,
           0.14679849]],
 
        [[-0.48547882, -0.6672664 , -0.30831295,  0.31500554,
           0.21519628, -0.09552038, -0.33133912,  0.8978053 ,
          -0.38653338]]], dtype=float32),
 array([[[-0.04071528, -0.5614164 , -0.5472108 ,  0.32481748,
          -0.1586527 ,  0.06802678, -0.04312551, -1.0716503 ,
          -0.09168857]],
 
        [[ 0.2695058 , -1.5634912 ,  0.20198631, -0.20597708,
           0.06802317,  0.2522897 , -0.01191515,  0.5282674 ,
          -0.3303873 ]]], dtype=float32),
 array([[[-0.1229878 ,  0.03982117, -0.29502654, -1.2658846 ,
          -0.3401566 , -0.93464977,  1.1978221 , -0.24731389,
          -0.51143396]],
 
        [[-0.05537069, -1.3277841 , -0.7294856 , -0.740143  ,
          -0.83612764,  0.6652604 ,  0.0570471 , -0.4578703 ,
          -0.36179888]]], dtype=float32),
 array([[1.084761 , 1.0133266]], dtype=float32)]